In [1]:
!pip install stanza

  Using cached stanza-1.8.1-py3-none-any.whl (970 kB)


In [2]:
import stanza
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

2024-04-05 06:42:32 INFO: Downloaded file to /home/zuoyu916/stanza_resources/resources.json
2024-04-05 06:42:32 INFO: Downloading default packages for language: en (English) ...


2024-04-05 06:42:35 INFO: Downloaded file to /home/zuoyu916/stanza_resources/en/default.zip
2024-04-05 06:42:39 INFO: Finished downloading models and saved to /home/zuoyu916/stanza_resources
2024-04-05 06:42:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-05 06:42:40 INFO: Downloaded file to /home/zuoyu916/stanza_resources/resources.json
2024-04-05 06:42:41 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-04-05 06:42:41 INFO: Using device: cpu
2024-04-05 06:42:41 INFO: Loading: tokenize
2024-04-05 06:42:45 INFO: Loading: mwt
2024-04-05 06:42:45 INFO: Loading: pos
2024-04-05 06:42:46 INFO: Loading: lemma
2024-04-05 06:42:46 INFO: Loading: constituency
2024-04-05 06:42:46 INFO: Loading: depparse
2024-04-05 06:42:47 INFO: Loading: sentiment
2024-04-05 06:42:48 INFO: Loading: ner

('Barack', 4, 'nsubj:pass')
('Obama', 1, 'flat')
('was', 4, 'aux:pass')
('born', 0, 'root')
('in', 6, 'case')
('Hawaii', 4, 'obl')
('.', 4, 'punct')


In [193]:
import stanza
#assuming NLP pipeline for english has already been initialised

def extract_words(input_text):

    # Process the input text
    doc = nlp(input_text)

    # Extract nouns (NN, NNS, NNP, NNPS)
    ##ADJ for adjectives
    ##N for nouns
    nouns = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos.startswith('N') or word.upos == "PROPN":
                print(f"{word.text}, {word.upos}, {word.deprel}")
            else:
                print(f"\t{word.text}, {word.upos}, {word.deprel}")

# Example usage
input_sentence = "The magnetic flux density is defined as the force per unit length per unit current acting on a straight current-carrying conductor placed perpendicular to a magnetic field."
nouns_list = extract_words(input_sentence)

	The, DET, det
	magnetic, ADJ, amod
flux, NOUN, compound
density, NOUN, nsubj:pass
	is, AUX, aux:pass
	defined, VERB, root
	as, ADP, case
	the, DET, det
force, NOUN, obl
	per, ADP, case
unit, NOUN, compound
length, NOUN, nmod
	per, ADP, case
unit, NOUN, compound
current, NOUN, nmod
	acting, VERB, advcl
	on, ADP, case
	a, DET, det
	straight, ADJ, amod
	current, ADJ, amod
	-, PUNCT, punct
	carrying, VERB, amod
conductor, NOUN, obl
	placed, VERB, acl
	perpendicular, ADJ, xcomp
	to, ADP, case
	a, DET, det
	magnetic, ADJ, amod
field, NOUN, obl
	., PUNCT, punct


In [65]:
def extract_subj(input_text):
    doc = nlp(input_text)
    long = []
    for sentence in doc.sentences:
        out = ""
        for word in sentence.words:
            if word.upos == "ADJ":
                out += f"{word.text} "
            elif word.upos.startswith("N"):
                out += f"{word.text} "
            if word.upos == "AUX":
                long.append(out)
                break
    if long:
        return long

['dog ']


In [135]:
input_sentence = """
The naturalistic fallacy: Just because this is the way things CURRENTLY ARE does not mean this is the way things OUGHT TO BE.
"""
print(extract_subj(input_sentence))

['naturalistic fallacy ']


In [215]:
import re
#Assuming stanza is imported
def extract_subj_v2(input_text):
    #Assuming stanza pipeline for english has already been initialised
    long_linetext = re.sub(r'[\n\r]+',': ',input_text)
    # print(long_linetext)
    doc = nlp(long_linetext)
    long = []
    for sentence in doc.sentences:
        out = ""
        for word in sentence.words:
            if word.upos == "ADJ":
                out += f"{word.text} "
            elif word.upos.startswith("N") or word.upos == "PROPN":
                out += f"{word.text} "
            elif word.upos == "AUX" or word.text==":":
                if out != '': 
                    long.append({"subj":out,word.upos:word.text})
                break

        # if out != '': long.append({"subj":out,word.upos:word.text})
    if long:
        return long

In [170]:
input_sentence = "A dog is a mammal."
print(extract_subj_v2(input_sentence))

[{'subj': 'dog ', 'AUX': 'is'}]


In [216]:
input_sentence = """The magnetic flux density is defined as the force per unit length per unit current acting
on a straight current-carrying conductor placed perpendicular to a magnetic field."""
# extract_words(input_sentence)
print(extract_subj_v2(input_sentence))

[{'subj': 'magnetic flux density ', 'AUX': 'is'}]


In [199]:
input_sentence = """Intuitionism: 
It is the theory that moral truths in metaethics are known directly by intuition."""
# extract_words(input_sentence)
print(extract_subj_v2(input_sentence))

[{'subj': 'Intuitionism ', 'PUNCT': ':'}]


In [168]:
input_sentence = """
The naturalistic fallacy: Just because this is the way things CURRENTLY ARE does not mean this is the way things OUGHT TO BE.
"""
print(extract_subj_v2(input_sentence))

[{'subj': 'naturalistic fallacy ', 'PUNCT': ':'}]


In [195]:
input_sentence = """Divine Command Theory (DCT)
It holds that all moral beliefs come from God."""
# extract_words(input_sentence)
print(extract_subj_v2(input_sentence))

	Divine, ADJ, amod
Command, PROPN, compound
Theory, PROPN, root
	(, PUNCT, punct
DCT, PROPN, appos
	), PUNCT, punct
	It, PRON, nsubj
	holds, VERB, root
	that, SCONJ, mark
	all, DET, det
	moral, ADJ, amod
beliefs, NOUN, nsubj
	come, VERB, ccomp
	from, ADP, case
God, PROPN, obl
	., PUNCT, punct
[{'subj': 'Divine Command Theory DCT ', 'PUNCT': ':'}]


In [224]:
input_sentence = """Force is the product of mass and acceleration. Momentum is the product of force and velocity."""
# extract_words(input_sentence) ##Able to distinguish the key points from 2 dff sentences
print(extract_subj_v2(input_sentence))

[{'subj': 'Force ', 'AUX': 'is'}, {'subj': 'Momentum ', 'AUX': 'is'}]


In [223]:
input_sentence = """Momentum is the product of force and velocity. It is the integral of Force w.r.t. time."""
# extract_words(input_sentence)
print(extract_subj_v2(input_sentence))

[{'subj': 'Momentum ', 'AUX': 'is'}]


In [227]:
input_sentence = """There is a high number of commuters in the evening, which is the cause of the jam."""
extract_words(input_sentence) ##Code understands that 'is' is not used for deginition in this case
print(extract_subj_v2(input_sentence))

	There, PRON, expl
	is, VERB, root
	a, DET, det
	high, ADJ, amod
number, NOUN, nsubj
	of, ADP, case
commuters, NOUN, nmod
	in, ADP, case
	the, DET, det
evening, NOUN, obl
	,, PUNCT, punct
	which, PRON, nsubj
	is, AUX, cop
	the, DET, det
cause, NOUN, acl:relcl
	of, ADP, case
	the, DET, det
jam, NOUN, nmod
	., PUNCT, punct
[{'subj': 'high number commuters evening ', 'AUX': 'is'}]
